In [36]:
import os
import json
import pandas as pd
import traceback


In [37]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [38]:
import openai
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key = api_key, model_name = "gpt-4o", temperature = 0.5)
# higher the value temperature, more creative is the model

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [40]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [41]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [42]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [43]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [44]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [45]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"], template = TEMPLATE
)

In [46]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [14]:
## Now, we'll connect our quiz chain and our review chain

In [47]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"], output_variables = ["quiz","review"], verbose = True )

In [48]:
file_path = r"C:\Users\aakri\mcqgen_proj1\data.txt"

In [49]:
with open(file_path, 'r') as file:
    text = file.read()

In [50]:
import json
json.dumps(RESPONSE_JSON)

#to convert python dict to json string

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [51]:
number = 5
subject = "HVAC"
tone = "simple"

In [52]:
# setting up token-usage-tracking

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : text,
            "number" : number,
            "subject" : subject,
            "tone" : tone,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field's abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).

HVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated with respect to temp

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field's abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).

HVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated w

In [53]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1915
Prompt Tokens:1106
Completion Tokens:809
Total Cost:0.017665


In [58]:
response

{'text': 'Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field\'s abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).\n\nHVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated with respect to temperature and humidi

In [56]:
print(quiz)

```json
{
  "1": {
    "mcq": "What does HVAC stand for?",
    "options": {
      "a": "Heating, Ventilation, and Air Conditioning",
      "b": "Heating, Vacuum, and Air Circulation",
      "c": "Heat, Ventilation, and Air Circulation",
      "d": "Heating, Ventilation, and Air Cleaning"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Which of the following is sometimes added to the HVAC abbreviation?",
    "options": {
      "a": "Refrigeration",
      "b": "Radiation",
      "c": "Regulation",
      "d": "Rehydration"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "What is the main goal of an HVAC system?",
    "options": {
      "a": "To provide thermal comfort and acceptable indoor air quality",
      "b": "To reduce energy consumption",
      "c": "To increase building security",
      "d": "To improve outdoor air quality"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "Which of the following is a building ventilation method mentioned in the text?",
    "options": {
  

In [69]:
quiz = quiz.strip()

# Remove Markdown code block if present
if quiz.startswith("```json"):
    quiz = quiz[7:]  # remove the ```json
if quiz.endswith("```"):
    quiz = quiz[:-3]  # remove the trailing ```



In [70]:
quiz

'\n{\n  "1": {\n    "mcq": "What does HVAC stand for?",\n    "options": {\n      "a": "Heating, Ventilation, and Air Conditioning",\n      "b": "Heating, Vacuum, and Air Circulation",\n      "c": "Heat, Ventilation, and Air Circulation",\n      "d": "Heating, Ventilation, and Air Cleaning"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "Which of the following is sometimes added to the HVAC abbreviation?",\n    "options": {\n      "a": "Refrigeration",\n      "b": "Radiation",\n      "c": "Regulation",\n      "d": "Rehydration"\n    },\n    "correct": "a"\n  },\n  "3": {\n    "mcq": "What is the main goal of an HVAC system?",\n    "options": {\n      "a": "To provide thermal comfort and acceptable indoor air quality",\n      "b": "To reduce energy consumption",\n      "c": "To increase building security",\n      "d": "To improve outdoor air quality"\n    },\n    "correct": "a"\n  },\n  "4": {\n    "mcq": "Which of the following is a building ventilation method mentioned in the 

In [66]:
quiz = response.get("quiz")

In [71]:
quiz = json.loads(quiz)

In [73]:
quiz_table_data = []
for key,value in quiz.items():
    mcq=value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append(
        {
            "MCQ": mcq,
            "Options": options,
            "Correct": correct
        }
    )

In [74]:
quiz_table_data

[{'MCQ': 'What does HVAC stand for?',
  'Options': 'a : Heating, Ventilation, and Air Conditioning | b : Heating, Vacuum, and Air Circulation | c : Heat, Ventilation, and Air Circulation | d : Heating, Ventilation, and Air Cleaning',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is sometimes added to the HVAC abbreviation?',
  'Options': 'a : Refrigeration | b : Radiation | c : Regulation | d : Rehydration',
  'Correct': 'a'},
 {'MCQ': 'What is the main goal of an HVAC system?',
  'Options': 'a : To provide thermal comfort and acceptable indoor air quality | b : To reduce energy consumption | c : To increase building security | d : To improve outdoor air quality',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is a building ventilation method mentioned in the text?',
  'Options': 'a : Mechanical (forced) | b : Electrical | c : Chemical | d : Magnetic',
  'Correct': 'a'},
 {'MCQ': 'In which environments is HVAC important according to the text?',
  'Options': 'a : Residential 

In [75]:
quiz=pd.DataFrame(quiz_table_data)

In [76]:
quiz.to_csv("HVAC.csv", index=False)